In [38]:
import pandas as pd
import numpy as np
import yfinance as yf
import cufflinks as cf

In [39]:
cf.go_offline()

In [40]:
tickerSymbol = 'WEGE3.SA'
tickerData = yf.Ticker(tickerSymbol)

In [41]:
tickerQuarterlyFinancials = tickerData.quarterly_financials
tickerQuarterlyFinancials.T.head()

,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,...,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares
,,,,,,,,,,,,,,,,,,,,,
2021-03-31,None,None,9.03717e+08,4.00951e+08,7.64258e+08,6.09209e+08,1.61974e+09,8.93182e+08,8.93182e+08,1.12924e+08,...,None,None,1.24796e+08,5.07688e+09,4.1837e+09,3.45714e+09,1.0535e+07,None,7.78921e+08,7.64258e+08
2020-12-31,None,None,8.69659e+08,3.67133e+08,7.42229e+08,5.89413e+08,1.5963e+09,8.59185e+08,8.59185e+08,1.43456e+08,...,None,None,1.07806e+08,4.88992e+09,4.03073e+09,3.29362e+09,1.0474e+07,None,7.61853e+08,7.42229e+08
2020-09-30,None,None,7.85504e+08,2.77565e+08,6.44246e+08,5.64902e+08,1.51157e+09,8.10636e+08,8.10636e+08,1.36034e+08,...,None,None,1.31892e+08,4.80126e+09,3.99062e+09,3.28969e+09,-2.5132e+07,None,6.53612e+08,6.44246e+08
2020-06-30,None,None,5.73301e+08,2.66371e+08,5.14375e+08,5.02193e+08,1.2321e+09,6.23696e+08,6.23696e+08,1.06214e+08,...,None,None,4.6784e+07,4.06394e+09,3.44025e+09,2.83184e+09,-5.0395e+07,None,5.26517e+08,5.14375e+08


In [93]:
tickerFinancials = tickerData.financials
tickerFinancials = tickerFinancials.T
tickerFinancials.index.year

Int64Index([2020, 2019, 2018, 2017], dtype='int64', name='')

In [91]:
import plotly.offline as pyoff
import plotly.graph_objs as go

data = [go.Bar(x=tickerFinancials.index.year,
               y=tickerFinancials['Total Revenue'],
               marker={'color': 'blue'}),
        
       go.Bar(x=tickerFinancials.index,
              y=tickerFinancials['Gross Profit'],
              marker={'color': 'orange'}),
        
       go.Bar(x=tickerFinancials.index,
               y=tickerFinancials['Ebit'],
               marker={'color': 'gray'}),
        
       go.Bar(x=tickerFinancials.index,
               y=tickerFinancials['Net Income'],
               marker={'color': 'yellow'})]

layout = go.Layout(title='Yearly Reports',
                  xaxis={'title':'Time'},
                  plot_bgcolor='black',
                  paper_bgcolor='black')

fig = go.Figure(data=data, layout=layout)
pyoff.iplot(fig)